In [ ]:
import pandas as pd
import sqlite3
print(pd.__version__)

In [ ]:
# ==================== Import modules & initialize DB ==================== #

import app_database as db
import importlib                                            # To reload the modules
importlib.reload(db)                                        # Reload the db module
import os
import datetime

db.drop_all_tables()
db.run_query("PRAGMA foreign_keys = ON;")

## ---------- Log in and Sessions ---------- ##

In [ ]:
db.create_table(
    'users',
    "id INTEGER PRIMARY KEY AUTOINCREMENT,"                    # Internal unique ID (Primary Key)
    "username TEXT UNIQUE,"                                    # Unique username
    "password TEXT,"                                           # Password (should be hashed)
    "email TEXT UNIQUE,"                                       # Unique email address
    "SSO_id INTEGER UNIQUE,"                                   # External SSO authentication ID
    "registration_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP"    # Account registration date
)

list_of_users = (
    ["admin", "admin", "admin@gevernova.com", 101010, datetime.datetime.now()],
    ["user1", "user1", "user1@gevernova.com", 111111, datetime.datetime.now()],
    ["user2", "user2", "user2@gevernova.com", 222222, datetime.datetime.now()],
    ["user3", "user3", "user3@gevernova.com", 333333, datetime.datetime.now()],
)
for data in list_of_users:
    db.add_element_to_table('users', data, False)

db.print_table_by_name('users')

In [ ]:
db.create_table(
    'users_sessions',
    "id INTEGER PRIMARY KEY AUTOINCREMENT,"                       # Session ID (Primary Key)
    "user_id INTEGER REFERENCES users(id) ON DELETE CASCADE,"     # Linked user ID (Foreign Key)
    "session_token TEXT UNIQUE,"                                  # Unique session token
    "is_active BOOLEAN DEFAULT true,"                             # Session status (active/inactive)
    "session_start TIMESTAMP DEFAULT CURRENT_TIMESTAMP,"          # Session creation timestamp
    "last_activity TIMESTAMP"                                     # Last user activity timestamp
)

list_of_sessions = (
    [101010, "tokenAdmin", True, datetime.datetime.now(), datetime.datetime.now()],
    [111111, "token1", True, datetime.datetime.now(), datetime.datetime.now()],
    [222222, "token2", True, datetime.datetime.now(), datetime.datetime.now()],
    [333333, "token3", True, datetime.datetime.now(), datetime.datetime.now()],
)
for data in list_of_sessions:
    db.add_element_to_table('users_sessions', data, False)

db.print_table_by_name('users_sessions')

In [ ]:
db.create_table(
    'login_history',
    "id INTEGER PRIMARY KEY AUTOINCREMENT,"                      # Event ID (Primary Key)
    "user_id INTEGER REFERENCES users(id),"                      # User who logged in (Foreign Key)
    "login_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP,"            # Login timestamp
    "logout_time TIMESTAMP"                                      # Logout timestamp (nullable)
)

list_of_logins = (
    [101010, datetime.datetime.now()-datetime.timedelta(days=1), datetime.datetime.now()],
    [111111, datetime.datetime.now()-datetime.timedelta(days=1), datetime.datetime.now()],
    [222222, datetime.datetime.now()-datetime.timedelta(days=1), datetime.datetime.now()],
    [333333, datetime.datetime.now()-datetime.timedelta(days=1), datetime.datetime.now()],
)
for data in list_of_logins:
    db.add_element_to_table('login_history', data, False)

db.print_table_by_name('login_history')

## ---------- Projects ---------- ##

In [ ]:
db.create_table(
    'projects',
    "id INTEGER PRIMARY KEY AUTOINCREMENT,"                      # Project ID (Primary Key)
    "title TEXT,"                                                # Project title
    "description TEXT,"                                          # Detailed project description
    "status TEXT,"                                               # Project status (e.g. Open, In Progress, Closed)
    "creation_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,"         # Date of project creation
    "last_update TIMESTAMP DEFAULT CURRENT_TIMESTAMP"            # Last project update timestamp
)

list_of_projects = (
    ["Project 1", "Description of the project 1", "Open", datetime.datetime.now(), datetime.datetime.now()],
    ["Project 2", "Description of the project 2", "Open", datetime.datetime.now(), datetime.datetime.now()],
    ["Project 3", "Description of the project 3", "Open", datetime.datetime.now(), datetime.datetime.now()],
    ["Project 4", "Description of the project 4", "Open", datetime.datetime.now(), datetime.datetime.now()],
)
for data in list_of_projects:
    db.add_element_to_table('projects', data, False)

db.print_table_by_name('projects')

In [ ]:
db.create_table(
    'comments',
    "id INTEGER PRIMARY KEY AUTOINCREMENT,"                             # Comment ID (Primary Key)
    "user_id INTEGER REFERENCES users(id) ON DELETE CASCADE,"           # Author of the comment (FK)
    "project_id INTEGER REFERENCES projects(id) ON DELETE CASCADE,"     # Target project (FK)
    "content TEXT,"                                                     # Comment text/content
    "creation_date TEXT"                                                # Timestamp of comment creation
)

list_of_comments = (
    [1, 1, "Comment 1", datetime.datetime.now()],
    [2, 2, "Comment 2", datetime.datetime.now()],
    [3, 3, "Comment 3", datetime.datetime.now()],
    [4, 4, "Comment 4", datetime.datetime.now()],
)
for data in list_of_comments:
    db.add_element_to_table('comments', data, False)

db.print_table_by_name('comments')

In [ ]:
db.create_table(
    'projects_members',
    "id INTEGER PRIMARY KEY AUTOINCREMENT,"                             # Membership ID (Primary Key)
    "project_id INTEGER REFERENCES projects(id) ON DELETE CASCADE,"     # Related project (FK)
    "user_id INTEGER REFERENCES users(id) ON DELETE CASCADE,"           # Member user (FK)
    "role TEXT,"                                                        # Role in the project (e.g. admin, contributor)
    "add_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP"                      # When the user was added to the project
)

list_of_projects_members = (
    [1, 1, "ApporteLesCroissants", datetime.datetime.now()],
    [2, 2, "ParleTrop", datetime.datetime.now()],
    [3, 3, "TheBoss", datetime.datetime.now()],
    [4, 4, "Nothing", datetime.datetime.now()],
)
for data in list_of_projects_members:
    db.add_element_to_table('projects_members', data, False)

db.print_table_by_name('projects_members')

## ---------- Products and Orders ---------- ##

In [ ]:
db.create_table(
    'products',
    "id INTEGER PRIMARY KEY AUTOINCREMENT,"     # Product ID (Primary Key)
    "reference INTEGER UNIQUE,"                 # Internal product reference code
    "name TEXT,"                                # Product name
    "description TEXT"                          # Product description
)

list_of_products = (
    [10000, "Banana", "Fruit"],
    [10000, "test", "test"],
    [22222, "Orange", "Fruit"],
    [55555, "Cucumber", "Vegetable"],
)
for data in list_of_products:
    try:
        db.add_element_to_table('products', data, True)
    except sqlite3.IntegrityError:
        print(f"⚠ Skipping duplicate product: {data}")

db.print_table_by_name('products')

In [ ]:
db.create_table(
    'product_options',
    "id INTEGER PRIMARY KEY AUTOINCREMENT,"     # Option ID (Primary Key)
    "product_id INTEGER,"                       # Product ID (FK, ideally should reference products(id))
    "category TEXT,"                            # Option category (e.g. color, size, weight)
    "option_code TEXT,"                         # Option code (e.g. red, XL, 1kg)
    "description TEXT"                          # Option description
)

list_of_product_options = (
    [10000, "color", "yellow", "Yellow banana"],
    [11111, "color", "red", "Red apple"],
    [11111, "color", "green", "Green apple"],
)
for data in list_of_product_options:
    db.add_element_to_table('product_options', data, True)

db.print_table_by_name('product_options')

In [ ]:
db.create_table(
    'options_rules',
    "id INTEGER PRIMARY KEY AUTOINCREMENT,"     # Rule ID (Primary Key)
    "product_id INTEGER,"                       # Product ID (FK, ideally should reference products(id))
    "option_code TEXT,"                         # Option code (e.g. red, XL, 1kg)
    "expression TEXT,"                          # Expression to evaluate the rule
    "rule_type TEXT"                            # Type of rule (e.g. required, forced, forbidden)
)

list_of_options_rules = (
    [1, "yellow", "A AND (B OR C)", "required"],
    [2, "red", "X AND (Y OR Z)", "forbidden"],
    [3, "green", "L AND (M OR N)", "forbidden"],
)
for data in list_of_options_rules:
    db.add_element_to_table('options_rules', data, False)

db.print_table_by_name('options_rules')

In [ ]:
db.create_table(
    'country_avaibility',
    "id INTEGER PRIMARY KEY AUTOINCREMENT,"     # ID (Primary Key)
    "product_id INTEGER,"                       # Product ID (FK, ideally should reference products(id))
    "country_code TEXT,"                         # Country code where the product is available
    "restriction_status TEXT"                   # Status of the product in the country (e.g. available, restricted, banned)
)

list_of_country_avaibility = (
    [10000, "FR", "available"],
    [10000, "US", "restricted"],
    [11111, "FR", "available"],
    [11111, "US", "available"],
    [22222, "FR", "available"],
    [22222, "US", "available"],
    [33333, "FR", "available"],
    [33333, "US", "available"],
    [44444, "FR", "available"],
    [44444, "US", "available"],
    [55555, "FR", "available"],
    [55555, "US", "restricted"],
    [66666, "FR", "restricted"],
    [66666, "US", "available"],
    [77777, "FR", "available"],
    [77777, "US", "banned"],
)
for data in list_of_country_avaibility:
    db.add_element_to_table('country_avaibility', data, True)

db.print_table_by_name('country_avaibility')

In [ ]:
db.create_table(
    'project_selected_options',
    "id INTEGER PRIMARY KEY AUTOINCREMENT,"             # Price ID (Primary Key)
    "project_id INTEGER,"                               # Project ID (FK, ideally should reference projects(id))
    "product_id INTEGER,"                               # Product ID (FK, ideally should reference products(id))
    "option_code TEXT,"                                 # Option code (e.g. red, XL, 1kg)
    "is_selected BOOLEAN,"                               # Is the option selected for the project
    "last_update TIMESTAMP DEFAULT CURRENT_TIMESTAMP"   # Last project update timestamp
)

list_of_project_selected_options = (
    [1, 10000, "yellow", True, datetime.datetime.now()],
    [2, 11111, "red", True, datetime.datetime.now()],
    [3, 22222, "green", True, datetime.datetime.now()],
    [4, 33333, "yellow", True, datetime.datetime.now()],
)
for data in list_of_project_selected_options:
    db.add_element_to_table('project_selected_options', data, False)

db.print_table_by_name('project_selected_options')

In [ ]:
db.create_table(
    'project_finalized_options',
    "id INTEGER PRIMARY KEY AUTOINCREMENT,"             # Price ID (Primary Key)
    "project_id INTEGER,"                               # Project ID (FK, ideally should reference projects(id))
    "product_id INTEGER,"                               # Product ID (FK, ideally should reference products(id))
    "option_code TEXT,"                                 # Option code (e.g. red, XL, 1kg)
    "is_selected BOOLEAN,"                              # Is the option selected for the project
    "finalized_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP"  # Last project update timestamp
)

list_of_project_selected_options = (
    [1, 10000, "yellow", True, datetime.datetime.now()],
    [2, 11111, "red", True, datetime.datetime.now()],
    [3, 22222, "green", True, datetime.datetime.now()],
    [4, 33333, "yellow", True, datetime.datetime.now()],
)
for data in list_of_project_selected_options:
    db.add_element_to_table('project_selected_options', data, False)

db.print_table_by_name('project_selected_options')

In [ ]:
db.create_table(
    'orders',
    "id INTEGER PRIMARY KEY AUTOINCREMENT,"     # Order ID (Primary Key)
    "user_id INTEGER,"                          # Customer (FK, ideally should reference users(id))
    "product_id INTEGER,"                       # Product ordered (FK, ideally should reference products(id))
    "project_id INTEGER,"                       # Project linked to the order (FK, ideally should reference projects
    "quantity INTEGER,"                         # Quantity ordered
    "order_date TEXT"                           # Date of the order
)

# ----- Function to read the instructions from a file ----- #
#? nothing to do here

# products is the table that contains the products

In [ ]:
#* Display the content of all the tables
#db.print_all_tables()

In [ ]:
def read_excel_file(file_path):
    # Load the Excel file
    df = pd.read_excel(file_path, header=None)
    
    # Display the initial DataFrame
    display(df)
    
    # Identify columns that have non-null values in the first row
    first_row = df.iloc[0]
    columns_with_data = first_row.notnull()
    
    # Keep only columns with non-null values in the first row
    df = df.loc[:, columns_with_data]
    display(df)
    
    # Set the first row as column headers
    df.columns = df.iloc[0]
    df = df.iloc[1:]  # Remove the first row which is now the header
    
    # Further filter rows based on non-null values in all columns
    df = df.dropna(how='all')
    
    # Display the processed DataFrame
    display(df)

    return df



In [ ]:
file_path = "C:/Users/223133654/Documents/Application/power-meter-selector/data/raw/GE Grid Automation EPM Meters Order Codes - 03-11-2025.xlsx"
df = read_excel_file(file_path)